**INFO7390 - ASSIGNMENT 2 - Crash Course in Generative AI Worked Examples**

Amrutha Edara -002776773

Niharika Santhoshini Karri - 002727629

# Fine-Tune a Generative AI Model for Dialogue Summarization

#### Setting-up Kernel

In [ ]:
# %pip install --upgrade pip
# %pip install --disable-pipe-version-check torch==1.13.1 torchdata==0.5.1 --quiet

# %pip install transformers==4.27.2 datasets==2.11.0 evaluate==0.4.0 rouge_score==0.1.2 loralib==0.1.1 peft==0.3.0 --quiet
%pip install --upgrade transformers accelerate einops xformers evaluate datasets rouge_score loralib peft

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 25.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.1/109.1 MB 10.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.1/519.1 kB 29.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 1.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 56.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 46.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 84.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━

In [ ]:
# %pip freeze

### 1.2 Importing Necessary Libraries

In [ ]:
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

###Importing the required models and tokenizer for further analysis

In [ ]:
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, GenerationConfig, TrainingArguments, Trainer
import torch
import time
import evaluate
import pandas as pd
import numpy as np

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

### 1.3 Loading Dataset

In [ ]:
huggingface_dataset_name = "knkarthick/dialogsum"

dataset = load_dataset(huggingface_dataset_name)

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [ ]:
model_name = "google/flan-t5-large"
device = "cuda" if torch.cuda.is_available() else "cpu"

#initializes a sequence-to-sequence language model and tokenizer from a pre-trained model

original_model = AutoModelForSeq2SeqLM.from_pretrained(model_name, torch_dtype=torch.bfloat16).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

####To find out the number of trainable parameters.



In [ ]:

#To calculates and returns the total and trainable parameters of a model, along with the percentage of trainable parameters.

def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()

    return f"Trainable model parameters: {trainable_model_params}\nAll model parameters: {all_model_params}\npercentage of trainable parameters: {(trainable_model_params/all_model_params)*100}"

In [ ]:
print(print_number_of_trainable_model_parameters(original_model))

Trainable model parameters: 247577856
All model parameters: 247577856
percentage of trainable parameters: 100.0


In [ ]:
print(print_number_of_trainable_model_parameters(original_model))

Trainable model parameters: 783150080
All model parameters: 783150080
percentage of trainable parameters: 100.0


#### 1.3 - Test the Model with Zero Shot Inferencing

Test the model with the zero shot inferencing. You can see that the model struggles to summarizes the dialogue compared to the baseline summary, but it does pull out some important information from the text which indicates the model can be fine-tuned to the task at hand.

In [ ]:
from transformers import pipeline
original_pipeline = pipeline(task="summarization", model=original_model, tokenizer=tokenizer, device=device)

In [ ]:
index = 200
dialogue = dataset['test'][index]['dialogue']
summary = dataset['test'][index]['summary']

prompt = f"""
Summarize the following conversation.

{dialogue}

Summary:
"""

output = original_pipeline(prompt)[0]['summary_text']

dash_line = "-".join('' for x in range(100))
print(dash_line)
print(f"INPUT PROMPT:\n{prompt}")
print(dash_line)
print(f"BASELINE HUMAN SUMMARY:\n{summary}")
print(dash_line)
print(f"MODEL GENERATION - ZERO SHOT:\n{output}\n")

---------------------------------------------------------------------------------------------------
INPUT PROMPT:

Summarize the following conversation.

#Person1#: Have you considered upgrading your system?
#Person2#: Yes, but I'm not sure what exactly I would need.
#Person1#: You could consider adding a painting program to your software. It would allow you to make up your own flyers and banners for advertising.
#Person2#: That would be a definite bonus.
#Person1#: You might also want to upgrade your hardware because it is pretty outdated now.
#Person2#: How can we do that?
#Person1#: You'd probably need a faster processor, to begin with. And you also need a more powerful hard disc, more memory and a faster modem. Do you have a CD-ROM drive?
#Person2#: No.
#Person1#: Then you might want to add a CD-ROM drive too, because most new software programs are coming out on Cds.
#Person2#: That sounds great. Thanks.

Summary:

-------------------------------------------------------------------

### 2 - Perform Full Fine-Tuning

#### 2.1 - Preprocess the Dialog-Summary Dataset





In [ ]:
def tokenize_function(example):
    start_prompt = "Summarize the following conversation.\n\n"
    end_prompt = "\n\nSummary: "
    prompt = [start_prompt + dialogue + end_prompt for dialogue in example["dialogue"]]
    example["input_ids"] = tokenizer(prompt, padding="max_length", truncation=True, return_tensors="pt").input_ids
    example["labels"] = tokenizer(example["summary"], padding="max_length", truncation=True, return_tensors="pt").input_ids

    return example

In [ ]:
dataset['test'][2]

{'id': 'test_0_3',
 'dialogue': "#Person1#: Ms. Dawson, I need you to take a dictation for me.\n#Person2#: Yes, sir...\n#Person1#: This should go out as an intra-office memorandum to all employees by this afternoon. Are you ready?\n#Person2#: Yes, sir. Go ahead.\n#Person1#: Attention all staff... Effective immediately, all office communications are restricted to email correspondence and official memos. The use of Instant Message programs by employees during working hours is strictly prohibited.\n#Person2#: Sir, does this apply to intra-office communications only? Or will it also restrict external communications?\n#Person1#: It should apply to all communications, not only in this office between employees, but also any outside communications.\n#Person2#: But sir, many employees use Instant Messaging to communicate with their clients.\n#Person1#: They will just have to change their communication methods. I don't want any - one using Instant Messaging in this office. It wastes too much tim

In [ ]:
tokenize_function(dataset['test'][2])

{'id': 'test_0_3',
 'dialogue': "#Person1#: Ms. Dawson, I need you to take a dictation for me.\n#Person2#: Yes, sir...\n#Person1#: This should go out as an intra-office memorandum to all employees by this afternoon. Are you ready?\n#Person2#: Yes, sir. Go ahead.\n#Person1#: Attention all staff... Effective immediately, all office communications are restricted to email correspondence and official memos. The use of Instant Message programs by employees during working hours is strictly prohibited.\n#Person2#: Sir, does this apply to intra-office communications only? Or will it also restrict external communications?\n#Person1#: It should apply to all communications, not only in this office between employees, but also any outside communications.\n#Person2#: But sir, many employees use Instant Messaging to communicate with their clients.\n#Person1#: They will just have to change their communication methods. I don't want any - one using Instant Messaging in this office. It wastes too much tim

In [ ]:
# The dataset actually contains 3 diff splits: train, validation, test
# The tokenize_function code is handling all data across all splits in batches.
tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/12460 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

In [ ]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic', 'input_ids', 'labels'],
        num_rows: 12460
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic', 'input_ids', 'labels'],
        num_rows: 500
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic', 'input_ids', 'labels'],
        num_rows: 1500
    })
})

In [ ]:
!nvidia-smi

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Mon Aug  7 10:40:09 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.161.03   Driver Version: 470.161.03   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/

In [ ]:
!pwd

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
/kaggle/working


In [ ]:
tokenized_datasets = tokenized_datasets.remove_columns(['id', "topic", "dialogue", "summary"])

In [ ]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 12460
    })
    validation: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 500
    })
    test: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 1500
    })
})

Only for this lab purpose, you will subsample the dataset:

#### 2.2 - Fine-Tune the Model with Preprocessed Dataset

In [ ]:
!pwd

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
/kaggle/working


In [ ]:
# Before training the model, clear the gpu cache memory.
import torch
import gc
gc.collect()
torch.cuda.empty_cache()

### To set training parameters for a model, and initializes a trainer with datasets for training and evaluation.

In [ ]:
# Before training the model, clear the gpu cache memory.
import torch
torch.cuda.empty_cache()

output_dir = f"./dialogue-summary-training-{str(int(time.time()))}"
training_args = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=5,
    warmup_steps=500,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    weight_decay=0.01,
    logging_steps=10,
    evaluation_strategy='steps',
    eval_steps=100,
    save_steps=100,
    gradient_accumulation_steps=2,
    gradient_checkpointing=True,
    optim="adafactor",
    fp16=False,
    learning_rate=3e-4,
    max_steps=500,
    save_total_limit=2
)

trainer = Trainer(
    model=original_model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation']
)

In [ ]:
trainer.train()

TrainOutput(global_step=500, training_loss=5.08451220703125, metrics={'train_runtime': 23982.7822, 'train_samples_per_second': 1.334, 'train_steps_per_second': 0.021, 'total_flos': 7.366038370910208e+16, 'train_loss': 5.08451220703125, 'epoch': 2.56})

In [ ]:
model_name

'google/flan-t5-large'

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
tokenizer_pre = AutoTokenizer.from_pretrained(model_name, use_fast=True)
model_pre = AutoModelForSeq2SeqLM.from_pretrained("/kaggle/working/dialogue-summary-training-1691405014/checkpoint-500").to(device)
instruct_pipeline = pipeline(task="summarization", model=model_pre, tokenizer=tokenizer_pre, device=device)

#### 2.3 - Evaluate the Model Qualitatively (Human Evaluation)



In [ ]:
index = 1
dialogue = dataset['test'][index]['dialogue']
human_baseline_summary = dataset['test'][index]['summary']

prompt = f"""
Summarize the following conversation.

{dialogue}

Summary:
"""

# original_input_ids = tokenizer(prompt, return_tensors="pt").input_ids
# instruct_input_ids = tokenizer_pre(prompt, return_tensors="pt").input_ids

# original_model_outputs = original_model.generate(input_ids=original_input_ids, generation_config=GenerationConfig(max_new_tokens=200, num_beams=1))
# original_model_text_output = tokenizer.decode(original_model_outputs[0], skip_special_tokens=True)

# instruct_model_outputs = model_pre.generate(input_ids=instruct_input_ids, generation_config=GenerationConfig(max_new_tokens=200, num_beams=1))
# instruct_model_text_output = tokenizer_pre.decode(instruct_model_outputs[0], skip_special_tokens=True)
original_model_text_output = original_pipeline(prompt, max_new_tokens=200, num_beams=1)
instruct_model_text_output = instruct_pipeline(prompt, max_new_tokens=200, num_beams=1)

dash_line = "-".join('' for x in range(100))
print(dash_line)
print(f"INPUT PROMPT:\n{prompt}")
print(f"BASELINE HUMAN SUMMARY:\n{human_baseline_summary}")
print(dash_line)
print(f"ORIGINAL MODEL:\n{original_model_text_output[0]['summary_text']}")
print(dash_line)
print(f"INSTRUCT MODEL:\n{instruct_model_text_output[0]['summary_text']}")

Both `max_new_tokens` (=200) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=200) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


---------------------------------------------------------------------------------------------------
INPUT PROMPT:

Summarize the following conversation.

#Person1#: Ms. Dawson, I need you to take a dictation for me.
#Person2#: Yes, sir...
#Person1#: This should go out as an intra-office memorandum to all employees by this afternoon. Are you ready?
#Person2#: Yes, sir. Go ahead.
#Person1#: Attention all staff... Effective immediately, all office communications are restricted to email correspondence and official memos. The use of Instant Message programs by employees during working hours is strictly prohibited.
#Person2#: Sir, does this apply to intra-office communications only? Or will it also restrict external communications?
#Person1#: It should apply to all communications, not only in this office between employees, but also any outside communications.
#Person2#: But sir, many employees use Instant Messaging to communicate with their clients.
#Person1#: They will just have to change t

#### 2.4 - Evaluate the Model Quantitatively (with ROGUE)

The ROGUE metric helps quantify the validity of summarizations produced by models. It compares summarizations to a "baseline" summary which is usually created by a human. While it is not perfect, it does indicate the overall increase in summarization effectiveness that we have accomplished by fine-tuning.

The ROUGE metric (Recall-Oriented Understudy for Gisting Evaluation) evaluates text summaries by comparing them with reference summaries. It includes variants like ROUGE-N (N-gram overlap), ROUGE-L (longest common subsequence), and ROUGE-S (skip-bigram overlap). ROUGE calculates precision, recall, and F-scores, but mainly focuses on lexical overlap. It's widely used in summarization tasks, though it has limitations in assessing semantic equivalence and text coherence.

In [ ]:
rouge = evaluate.load("rouge")

In [ ]:
dialogues = dataset['test'][0:10]['dialogue']
human_baseline_summaries = dataset['test'][0:10]['summary']

In [ ]:
original_model_summaries = []
instruct_model_summaries = []

In [ ]:
for _, dialogue in enumerate(dialogues):
    prompt = f"""
Summarize the following conversation.

{dialogue}

Summary:
"""
    original_model_text_output = original_pipeline(prompt, max_new_tokens=200, num_beams=1)[0]['summary_text']
    instruct_model_text_output = instruct_pipeline(prompt, max_new_tokens=200, num_beams=1)[0]['summary_text']

    original_model_summaries.append(original_model_text_output)
    instruct_model_summaries.append(instruct_model_text_output)


Both `max_new_tokens` (=200) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=200) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=200) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=200) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both

In [ ]:
zipped_summaries = list(zip(human_baseline_summaries, original_model_summaries, instruct_model_summaries))
df = pd.DataFrame(zipped_summaries, columns=["human_baseline_summaries", "original_model_summaries", "instruct_model_summaries"])
df

human_baseline_summaries  \
0  Ms. Dawson helps #Person1# to write a memo to ...   
1  In order to prevent employees from wasting tim...   
2  Ms. Dawson takes a dictation for #Person1# abo...   
3  #Person2# arrives late because of traffic jam....   
4  #Person2# decides to follow #Person1#'s sugges...   
5  #Person2# complains to #Person1# about the tra...   
6  #Person1# tells Kate that Masha and Hero get d...   
7  #Person1# tells Kate that Masha and Hero are g...   
8  #Person1# and Kate talk about the divorce betw...   
9  #Person1# and Brian are at the birthday party ...   

                            original_model_summaries  \
0  #Person1# asks Ms. Dawson to take a dictation ...   
1  #Person1# asks Ms. Dawson to take a dictation ...   
2  #Person1# asks Ms. Dawson to take a dictation ...   
3  #Person2# got stuck in traffic again and #Pers...   
4  #Person2# got stuck in traffic again and #Pers...   
5  #Person2# got stuck in traffic again and #Pers...   
6  Masha and Hero are getting divorced. Kate can'...   
7  Masha and Hero are getting divorced. Kate can'...   
8  Masha and Hero are getting divorced. Kate can'...   
9  #Person1# comes to Brian's birthday party and ...   

                            instruct_model_summaries  
0  #Person1# asks Ms. Dawson to take dictation fo...  
1  #Person1# asks Ms. Dawson to take dictation fo...  
2  #Person1# asks Ms. Dawson to take dictation fo...  
3  #Person2# got stuck in traffic again and #Pers...  
4  #Person2# got stuck in traffic again and #Pers...  
5  #Person2# got stuck in traffic again and #Pers...  
6  Masha and Hero are getting divorced. Kate can'...  
7  Masha and Hero are getting divorced. Kate can'...  
8  Masha and Hero are getting divorced. Kate can'...  
9  #Person1# comes to Brian's birthday party and ...

In [ ]:
original_model_results = rouge.compute(
    predictions=original_model_summaries,
    references=human_baseline_summaries[0:len(original_model_summaries)],
    use_aggregator=True,
    use_stemmer=True
)

In [ ]:
original_model_results

{'rouge1': 0.358461038961039,
 'rouge2': 0.13355291186730134,
 'rougeL': 0.27057471023600055,
 'rougeLsum': 0.27292752408881443}

In [ ]:
instruct_model_results = rouge.compute(
    predictions=instruct_model_summaries,
    references=human_baseline_summaries[0:len(instruct_model_summaries)],
    use_aggregator=True,
    use_stemmer=True
)

In [ ]:
instruct_model_results

{'rouge1': 0.3820808580631425,
 'rouge2': 0.14505262893744827,
 'rougeL': 0.28464822318524063,
 'rougeLsum': 0.284967949114168}

For practical use case, use much larger dataset as well as fine tune for longer time.

### 3 - Perform Parameter Efficient Fine-Tuning (PEFT)

Now, let's perform Parameter Efficient Fine-Tuning (PEFT) fine-tuning as opposed to "full fine-tuning” as you did above, PEFT is a form of instruction fine-tuning that is much more efficient than full fine-tuning - with comparable evaluation results as you will see soon.

PEFT is a generic term that includes Low-Rank Adaptation (LoRA) and prompt tuning (which is NOT THE SAME as prompt engineering!). In most cases, when
someone says PEFT, they typically mean LoRA. LoRA, at a very high level, allows the user to fine-tune their model using fewer compute resources (in some cases, a single GPU). After fine-tuning for a specific task, use case, or tenant with LoRA, the result is that the original LLM remains unchanged and a newly-trained “LoRA adapter” emerges. This LoRA adapter is much, much smaller than the original LLM - on the order of a single-digit % of the original LLM size (MBs vs GBs).


#### 3.1 - Setup the PEFT/LoRA model for Fine-Tuning

You need to set up the PEFT/LoRA model for fine-tuning with a new layer/parameter adapter. Using PEFT/LoRA, you are freezing the underlying LLM and only training the adapter. Have a look at the LoRA configuration below. Note the rank ( r ) hyper-parameter, which defines the rank/dimension of the adapter to be trained.


In [ ]:
%pip install loralib==0.1.1 peft==0.3.0

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 1.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from peft import LoraConfig, get_peft_model, TaskType

In [ ]:
help(get_peft_model)

Help on function get_peft_model in module peft.mapping:

get_peft_model(model, peft_config)
    Returns a Peft model object from a model and a config.
    
    Args:
        model ([`transformers.PreTrainedModel`]): Model to be wrapped.
        peft_config ([`PeftConfig`]): Configuration object containing the parameters of the Peft model.



In [ ]:
help(TaskType)

Help on class TaskType in module peft.utils.config:

class TaskType(builtins.str, enum.Enum)
 |  TaskType(value, names=None, *, module=None, qualname=None, type=None, start=1)
 |  
 |  An enumeration.
 |  
 |  Method resolution order:
 |      TaskType
 |      builtins.str
 |      enum.Enum
 |      builtins.object
 |  
 |  Data and other attributes defined here:
 |  
 |  CAUSAL_LM = <TaskType.CAUSAL_LM: 'CAUSAL_LM'>
 |  
 |  SEQ_2_SEQ_LM = <TaskType.SEQ_2_SEQ_LM: 'SEQ_2_SEQ_LM'>
 |  
 |  SEQ_CLS = <TaskType.SEQ_CLS: 'SEQ_CLS'>
 |  
 |  TOKEN_CLS = <TaskType.TOKEN_CLS: 'TOKEN_CLS'>
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors inherited from enum.Enum:
 |  
 |  name
 |      The name of the Enum member.
 |  
 |  value
 |      The value of the Enum member.
 |  
 |  ----------------------------------------------------------------------
 |  Readonly properties inherited from enum.EnumMeta:
 |  
 |  __members__
 |      Returns a mapping o

In [ ]:
lora_config = LoraConfig(
    r=32, # Rank
    lora_alpha=32,
    target_modules=["q", "v"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.SEQ_2_SEQ_LM #FLAN-T5
)

In [ ]:
peft_model = get_peft_model(original_model, lora_config)
print(print_number_of_trainable_model_parameters(peft_model))

Trainable model parameters: 3538944
All model parameters: 251116800


#### 3.2 - Train PEFT Adapter

Define training arguments and Trainer instance.

In [ ]:
output_dir = f"./peft-dialogue-summary-training-{str(int(time.time()))}"
peft_training_args = TrainingArguments(
    output_dir=output_dir,
    learning_rate=1e-5, # Higher Learning rate than full fine-tuning
    num_train_epochs=1,
    weight_decay=0.01,
    logging_steps=1,
    max_steps=1
)

peft_trainer = Trainer(
    model=original_model,
    args=peft_training_args,
    train_dataset=tokenized_datasets['train'],
)

In [ ]:
peft_trainer.train()

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 trainer.train()                                                                              │
│   2                                                                                              │
│                                                                                                  │
│ /opt/conda/lib/python3.10/site-packages/transformers/trainer.py:1645 in train                    │
│                                                                                                  │
│   1642 │   │   inner_training_loop = find_executable_batch_size(                                 │
│   1643 │   │   │   self._inner_training_loop, self._train_batch_size, args.auto_find_batch_size  │
│   1644 │   │   )                                                                                 │
│ ❱ 1645 │   │   return inner_training_loop(                                                       │
│   1646 │   │   │   args=args,                                                                    │
│   1647 │   │   │   resume_from_checkpoint=resume_from_checkpoint,                                │
│   1648 │   │   │   trial=trial,                                                                  │
│                                                                                                  │
│ /opt/conda/lib/python3.10/site-packages/transformers/trainer.py:1938 in _inner_training_loop     │
│                                                                                                  │
│   1935 │   │   │   │   │   self.control = self.callback_handler.on_step_begin(args, self.state,  │
│   1936 │   │   │   │                                                                             │
│   1937 │   │   │   │   with self.accelerator.accumulate(model):                                  │
│ ❱ 1938 │   │   │   │   │   tr_loss_step = self.training_step(model, inputs)                      │
│   1939 │   │   │   │                                                                             │
│   1940 │   │   │   │   if (                                                                      │
│   1941 │   │   │   │   │   args.logging_nan_inf_filter                                           │
│                                                                                                  │
│ /opt/conda/lib/python3.10/site-packages/transformers/trainer.py:2759 in training_step            │
│                                                                                                  │
│   2756 │   │   │   return loss_mb.reduce_mean().detach().to(self.args.device)                    │
│   2757 │   │                                                                                     │
│   2758 │   │   with self.compute_loss_context_manager():                                         │
│ ❱ 2759 │   │   │   loss = self.compute_loss(model, inputs)                                       │
│   2760 │   │                                                                                     │
│   2761 │   │   if self.args.n_gpu > 1:                                                           │
│   2762 │   │   │   loss = loss.mean()  # mean() to average on multi-gpu parallel training        │
│                                                                                                  │
│ /opt/conda/lib/python3.10/site-packages/transformers/trainer.py:2784 in compute_loss             │
│                                                                                                  │
│   2781 │   │   │   labels = inputs.pop("labels")                                                 │
│   2782 │   │   else:                                       

Prepare this model by adding an adapter to the original FLAN-TS model. You are setting is_trainable=False because the plan is only to perform inference with this PEFT model. If you were preparing the model for further training, you would set is_trainable=True .


In [ ]:
from peft import PeftModel, PeftConfig

peft_model_base = AutoModelForSeq2SeqlM. from_pretrained("google/flan-t5-base", torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")

## Adding the LoRA Adapter
peft_model = PeftModel.from_pretrained(peft_model_base,'./path/where/peft/checkpoint-model/is/saved/', torch_dtype=torch.bfloatl6, is_trainable=False)

REFERENCE:

https://github.com/Ryota-Kawamura/Generative-AI-with-LLMs/tree/main

https://huggingface.co/docs/peft/index

https://github.com/microsoft/LoRA

###Licences:

MIT License


Copyright (c) 2023 Amrutha Sai Edara, Niharika Karri


Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:


The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.


THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE